# Quickdraw model training

In [4]:
import logging
import os

import tensorflow as tf

from datetime import datetime

In [14]:
# variables initialization
curr_date = datetime.now().strftime("%Y%m%d_%H%M%S")

BUCKET_NAME = '<TO DEFINE>'
GCS_TRAINING_DATA = f'gs://{BUCKET_NAME}/tfrecord_data/training_data/'
GCS_VALIDATION_DATA = f'gs://{BUCKET_NAME}/tfrecord_data/validation_data/'
GCS_MODEL_DATA_PATH = f'gs://{BUCKET_NAME}/gcs_model_data/quickdraw_classifier_{curr_date}/'

batch_size = 50
validation_batch_size = 20
training_ds_size = 25000
validation_ds_size = 5000
img_height = 64
img_width = 64
nb_classes = 5
lr = 0.001
num_epochs = 5

In [9]:
def parse_tfrecord(example, img_size):
    """
    Parse tf record that represents an image of size img_size,
    and contains additional information such as : class number, label and one hot encoded class
    :param example: TFRecord encoded image
    :param img_size: image size
    :return:
    """
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class_num": tf.io.FixedLenFeature([], tf.int64),
        "label": tf.io.FixedLenFeature([], tf.string),
        "one_hot_class": tf.io.VarLenFeature(tf.float32)
    }

    example = tf.io.parse_single_example(example, features)

    image = tf.io.decode_jpeg(example['image'], channels=1)
    image = tf.reshape(image, [*img_size, 1])

    class_num = example['class_num']
    label = example['label']
    one_hot_class = tf.sparse.to_dense(example['one_hot_class'])

    return image, class_num, label, one_hot_class

In [10]:
def get_img_tfrec_dataset(input_path: str, dataset_size: int, img_size: (int, int)):
    """
    Read a dataset from Google Cloud Storage and preprocess it
    :param input_path: input GCS path
    :param dataset_size: input dataset_size
    :param img_size: pair of image parameters (image height, image width)
    :return: shuffled dataset with (image, one_hot_class) parameters
    """
    option_no_order = tf.data.Options()
    option_no_order.experimental_deterministic = False
    AUTOTUNE = tf.data.AUTOTUNE

    filenames = tf.io.gfile.glob(input_path + "*.tfrec")

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE)
    dataset = dataset.with_options(option_no_order)
    dataset = dataset.map(lambda img: parse_tfrecord(img, img_size), num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(lambda image, class_num, label, one_hot_class: (image, one_hot_class))
    dataset = dataset.shuffle(dataset_size)

    return dataset

In [11]:
def create_model(img_height, img_width, nb_classes, lr):
    """
    Creates keras model
    :param img_height: image height
    :param img_width: image width
    :param nb_classes: number of classes
    :param lr: optimizer learning rate
    :return: model
    """
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Convolution2D(
        input_shape=(img_height, img_width, 1),
        kernel_size=5,
        filters=32,
        padding='same',
        activation=tf.keras.activations.relu
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

    model.add(tf.keras.layers.Convolution2D(
        kernel_size=3,
        filters=32,
        padding='same',
        activation=tf.keras.activations.relu,
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

    model.add(tf.keras.layers.Convolution2D(
        kernel_size=3,
        filters=64,
        padding='same',
        activation=tf.keras.activations.relu
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=512, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(units=nb_classes, activation=tf.keras.activations.softmax))

    rms_prop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)

    model.compile(
        optimizer=rms_prop_optimizer,
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=['accuracy']
    )

    return model

In [15]:
strategy = tf.distribute.get_strategy()

logging.info(f"Reading training dataset at {GCS_TRAINING_DATA}")
training_dataset = get_img_tfrec_dataset(GCS_TRAINING_DATA, training_ds_size, (img_height, img_width))
training_dataset = training_dataset.batch(batch_size)

logging.info(f"Reading validation dataset at {GCS_VALIDATION_DATA}")
validation_dataset = get_img_tfrec_dataset(GCS_VALIDATION_DATA, validation_ds_size, (img_height, img_width))
validation_dataset = validation_dataset.batch(validation_batch_size)

logging.info(f"Defining checkpoint and early stopping callbacks")
gcs_checkpoint_path = os.path.join(GCS_MODEL_DATA_PATH, "model", "")
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(gcs_checkpoint_path, save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=2, restore_best_weights=True
)
with strategy.scope():
    model = create_model(img_height, img_width, nb_classes, lr)

logging.info("Starting model fitting ...")
history = model.fit(
    training_dataset,
    epochs=num_epochs,
    validation_data=validation_dataset,
    callbacks=[checkpoint_cb, early_stopping_cb],
    verbose=1
)

# save metric for hyperparemeter tuning
#hp_metric = history.history['val_accuracy'][-1]
#
#hpt = hypertune.HyperTune()
#hpt.report_hyperparameter_tuning_metric(hyperparameter_metric_tag='val_accuracy',
#                                        metric_value=hp_metric,
#                                        global_step=num_epochs)

Epoch 1/5


2022-04-17 16:49:19.928862: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


    499/Unknown - 13s 7ms/step - loss: 3.2550 - accuracy: 0.6499

2022-04-17 16:49:30.859536: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://devoxx_poc/gcs_model_data/quickdraw_classifier_20220417_164912/model/assets


...

500/500 [==============================] - 20s 20ms/step - loss: 3.2496 - accuracy: 0.6504 - val_loss: 0.4681 - val_accuracy: 0.8490
Epoch 2/5
497/500 [============================>.] - ETA: 0s - loss: 0.2948 - accuracy: 0.9090INFO:tensorflow:Assets written to: gs://devoxx_poc/gcs_model_data/quickdraw_classifier_20220417_164912/model/assets


INFO:tensorflow:Assets written to: gs://devoxx_poc/gcs_model_data/quickdraw_classifier_20220417_164912/model/assets


500/500 [==============================] - 9s 16ms/step - loss: 0.2955 - accuracy: 0.9087 - val_loss: 0.2748 - val_accuracy: 0.9142
Epoch 3/5
497/500 [============================>.] - ETA: 0s - loss: 0.2079 - accuracy: 0.9352INFO:tensorflow:Assets written to: gs://devoxx_poc/gcs_model_data/quickdraw_classifier_20220417_164912/model/assets


INFO:tensorflow:Assets written to: gs://devoxx_poc/gcs_model_data/quickdraw_classifier_20220417_164912/model/assets


500/500 [==============================] - 9s 16ms/step - loss: 0.2085 - accuracy: 0.9350 - val_loss: 0.2086 - val_accuracy: 0.9352
Epoch 4/5
500/500 [==============================] - 5s 9ms/step - loss: 0.1718 - accuracy: 0.9468 - val_loss: 0.2906 - val_accuracy: 0.9288
Epoch 5/5
500/500 [==============================] - 5s 9ms/step - loss: 0.1456 - accuracy: 0.9545 - val_loss: 0.2113 - val_accuracy: 0.9396


In [25]:
from keras.preprocessing import image
import numpy as np
from operator import itemgetter

img = image.load_img("../testing_data/crown/crown_6004.png", color_mode='grayscale',target_size=[img_height, img_width, 1])
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
probabilities = model.predict(images, batch_size=1)[0]
img_classes = ['angel', 'cat', 'crown', 'The Eiffel Tower', 'The Mona Lisa']

probabilities_with_classes = zip(img_classes, probabilities)
sorted_probabilities = sorted(probabilities_with_classes,
                              key=itemgetter(1),
                              reverse=True)

print(sorted_probabilities)

[('crown', 0.9999999), ('cat', 9.150056e-08), ('The Mona Lisa', 7.506949e-11), ('angel', 4.0674842e-11), ('The Eiffel Tower', 4.4128125e-15)]
